In [49]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install optuna

   ---------------------------------------- 383.6/383.6 kB 7.9 MB/s eta 0:00:00
   --------------------------------------- 233.6/233.6 kB 14.9 MB/s eta 0:00:00
   ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.12.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install xgboost

   --------------------------------------- 124.9/124.9 MB 26.2 MB/s eta 0:00:00


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
import optuna
from imblearn.under_sampling import NearMiss
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import pathlib

# Load Data
DATA_PATH = pathlib.Path("C:/Ahsan.docx/PROJECT/MetroPT_project/MetroPT/")
df = pd.read_feather(DATA_PATH / "Classification.feather")

# Prepare Data
X = df.drop(columns=["timestamp", "Target"])  # Drop timestamp
y = df["Target"]

nm_undersampler = NearMiss(version=3, n_neighbors_ver3=3, n_jobs=-1) # Warning takes very long to run
X, y = nm_undersampler.fit_resample(X, y)

# Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42099, stratify=y)

# Normalize Data (Important for Logistic Regression & SVM)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


C:\Users\91770\anaconda3\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\91770\anaconda3\lib\site-packages\imblearn\under_sampling\_prototype_selection\_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(


In [60]:
# Objective Function for Logistic Regression
def objective_logistic(trial, X_train, X_val, y_train, y_val):
    params = {
        "C": trial.suggest_float("C", 0.0001,1000, log=True),
        "class_weight": "balanced",
        "penalty" : "l2",
        "max_iter": 1000,
        "random_state": 42099
    }

    # Create and train the model
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)

    # Predict probabilities
    preds = model.predict_proba(X_val)

    logloss_scores = []
    logloss_scores.append(log_loss(y_val, preds))

    return logloss_scores[-1]  # Return the latest score

In [68]:
# Objective Function for Random Forest
def objective_rf(trial, X_train, X_val, y_train, y_val):
    params = {
        "n_estimators": 200,
        "class_weight": "balanced",
        "min_samples_split": trial.suggest_float("min_samples_split",0.001, 0.01, log= True), 
        "min_samples_leaf": trial.suggest_float("min_samples_leaf",0.001, 0.01, log= True), 
        "max_features": "log2",
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "random_state": 42099
    }
        
    model = RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    
    preds = model.predict_proba(X_val)
    
    #     logloss_scores = []
    #     logloss_scores.append(log_loss(y_val, preds))

    return log_loss(y_val, preds)  # Return the latest score

In [93]:
# Objective Function for XGBoost
def objective_xgb(trial, X_train, X_val, y_train, y_val):
    params = {
        "objective": "multi:softmax",  # or "multi:softprob"
#         "num_class": 3,
        "eval_metric": "mlogloss",
        "n_estimators": 200,
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.3),
        "max_depth": trial.suggest_int("max_depth", 9, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 5, 8),
        "gamma": trial.suggest_loguniform("gamma", 0.01, 5.0),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0, log=True),
#         "reg_alpha": trial.suggest_loguniform("reg_alpha", 0.01, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 0.01, 10.0),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "random_state": 42099,
    }
    
    model = XGBClassifier(**params)

    model.fit(X_train, y_train)
    preds = model.predict_proba(X_val)
#     logloss_scores.append(log_loss(y_val, preds))

    return log_loss(y_val, preds)

In [76]:
class HyperparameterTuner:
    def __init__(self, objective_func, n_trials=50, pruner=None):
        self.objective_func = objective_func
        self.n_trials = n_trials
        self.pruner = pruner if pruner else optuna.pruners.MedianPruner()
        self.study = None

    def tune(self, X_train, X_val, y_train, y_val):
        """Runs hyperparameter tuning using Optuna"""
        def objective(trial):
            return self.objective_func(trial, X_train, X_val, y_train, y_val)
        
        self.study = optuna.create_study(direction="minimize", pruner=self.pruner)
        self.study.optimize(objective, n_trials=self.n_trials) 
        
        return self.study.best_params, self.study.best_value

    def get_best_params(self):
        """Retrieve best parameters after tuning"""
        return self.study.best_params if self.study else None

    def get_best_score(self):
        """Retrieve best score after tuning"""
        return self.study.best_value if self.study else None


In [62]:
# Tune Logistic Regression
tuner = HyperparameterTuner(objective_func=objective_logistic, n_trials=30)
best_params_logistic = tuner.tune(X_train, X_val, y_train, y_val)
print("Best Logistic Regression Params:", best_params_logistic)

[I 2025-02-21 17:00:01,179] A new study created in memory with name: no-name-be91013d-5f09-4728-b271-42c07c7499f0
[I 2025-02-21 17:00:01,568] Trial 0 finished with value: 0.9529519843001338 and parameters: {'C': 95.1271683229839}. Best is trial 0 with value: 0.9529519843001338.
[I 2025-02-21 17:00:01,931] Trial 1 finished with value: 0.9529085669980438 and parameters: {'C': 0.32151444848143884}. Best is trial 1 with value: 0.9529085669980438.
[I 2025-02-21 17:00:02,289] Trial 2 finished with value: 0.9529538428580157 and parameters: {'C': 0.2240858130329955}. Best is trial 1 with value: 0.9529085669980438.
[I 2025-02-21 17:00:02,678] Trial 3 finished with value: 0.9529523965147062 and parameters: {'C': 12.638943983698182}. Best is trial 1 with value: 0.9529085669980438.
[I 2025-02-21 17:00:02,901] Trial 4 finished with value: 0.9535152778638091 and parameters: {'C': 0.005853452343654696}. Best is trial 1 with value: 0.9529085669980438.
[I 2025-02-21 17:00:03,285] Trial 5 finished with 

Best Logistic Regression Params: ({'C': 0.32151444848143884}, 0.9529085669980438)


In [69]:
# Tune Random Forest
tuner = HyperparameterTuner(objective_func=objective_rf, n_trials=50)
best_params_rf = tuner.tune(X_train, X_val, y_train, y_val)
print("Best Random Forest Params:", best_params_rf)

[I 2025-02-21 18:38:48,826] A new study created in memory with name: no-name-550b7ca4-d18b-45f6-83f6-475d63f75776
[I 2025-02-21 18:38:52,804] Trial 0 finished with value: 0.6674836875328193 and parameters: {'min_samples_split': 0.005649947680359575, 'min_samples_leaf': 0.0011596787527683007, 'max_depth': 8}. Best is trial 0 with value: 0.6674836875328193.
[I 2025-02-21 18:38:56,739] Trial 1 finished with value: 0.6813540161330318 and parameters: {'min_samples_split': 0.00885430001392939, 'min_samples_leaf': 0.003742701665602881, 'max_depth': 9}. Best is trial 0 with value: 0.6674836875328193.
[I 2025-02-21 18:39:00,037] Trial 2 finished with value: 0.7377429011306188 and parameters: {'min_samples_split': 0.007132384998227272, 'min_samples_leaf': 0.0011641650301377256, 'max_depth': 6}. Best is trial 0 with value: 0.6674836875328193.
[I 2025-02-21 18:39:03,751] Trial 3 finished with value: 0.6946041019805849 and parameters: {'min_samples_split': 0.0034970366409850804, 'min_samples_leaf':

Best Random Forest Params: {'min_samples_split': 0.0030959784653249956, 'min_samples_leaf': 0.0011366757909473594, 'max_depth': 10}


In [94]:
# Tune XGBoost
tuner = HyperparameterTuner(objective_func=objective_xgb, n_trials=50)
best_params_xgb = tuner.tune(X_train, X_val, y_train, y_val)
print("Best XGBoost Params:", best_params_xgb)

[I 2025-02-28 15:48:15,924] A new study created in memory with name: no-name-f72859f5-ded0-4d61-a8d8-ce38681dccf7
[I 2025-02-28 15:48:16,868] Trial 0 finished with value: 0.37827885254750254 and parameters: {'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.37827885254750254.
[I 2025-02-28 15:48:17,781] Trial 1 finished with value: 0.37827885254750254 and parameters: {'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.37827885254750254.
[I 2025-02-28 15:48:18,633] Trial 2 finished with value: 0.37827885254750254 and parameters: {'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.37827885254750254.
[I 2025-02-28 15:48:19,867] Trial 3 finished with value: 0.37827885254750254 and parameters: {'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.37827885254750254.
[I 2025-02-28 15:48:21,234] Trial 4 finished with value: 0.37827885254750254 and parameters: {'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.37827885254750254.
[I 2025-02-28 15:48:22,326

Best XGBoost Params: ({'grow_policy': 'depthwise'}, 0.37827885254750254)
